In [1]:
import requests
import json
import pandas as pd

In [2]:
# get game id for majora's mask

games = requests.get('https://www.speedrun.com/api/v1/games?abbreviation=mm').text
game_dict = json.loads(games)['data'][0]

game_id = game_dict['id']
game_id

'268zey6p'

In [21]:
# get all runs done for mm (max 200 at a time)

runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&game=' + game_id).text
runs_dict1 = json.loads(runs)
runs_list = json.loads(runs)['data']
runs_dict1['pagination']['size'] # max elements



200

In [22]:
# next course of action:
# loop through each run (dict)
# get the run's player id and primary_t (time) add add to dictionary 'id':'time'

# how to get player id of run at list index 0
runs_list[0]['players'][0]['id']

# how to get time of run 0
runs_list[0]['times']['primary_t']

len(runs_list)

200

In [23]:
# create a function which loops through a page (200 runs) then adds those runs
# to a larger list and continues until all runs are in the massive list

def make_list():
  full_list = []
  offset = 0
  runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&game=' + game_id).text
  dict = json.loads(runs)

  while dict['pagination']['size'] == 200:
    runs = requests.get('https://www.speedrun.com/api/v1/runs?max=200&game=' + game_id + '&offset=' + str(offset)).text
    dict = json.loads(runs)
    full_list.extend(dict['data'])
    offset += 200

  return full_list

In [24]:
# create a method that converts an id to a username

def id_to_username(id):
  user_info = json.loads(requests.get('https://www.speedrun.com/api/v1/users/' + id).text)
  if user_info.get('data'): # not sure why data key errors are happening here
    username = user_info['data']['names']['international']
  else:
    return 'fail'

  return username

In [25]:
# create a function which loops through one page/request(to start) and adds the
# time of all runs to the user's id in dictionary
# note this method takes into account players who don't have src accounts and
# thus do not have an id, however they are not accounted for in the final return

ids_test = set()

# takes
def add_time():
  runs = make_list()
  player_times = {}
  id_names = {}
  final = {}
  ids = set()

  for i in range(len(runs)):
    if runs[i]['players'][0].get('id'):
      player = runs[i]['players'][0]['id']
      #player = id_to_username(player)
      ids.add(player)
    else:
      player = runs[i]['players'][0]['name']


    time = runs[i]['times']['primary_t']

    if player in player_times:
      player_times[player] += time
    else:
      player_times[player] = time

  # create a mapping of ids to usernames
  for id in ids:
    id_names[id] = id_to_username(id)

  for id in id_names.keys() & player_times.keys():
    final[id_names[id]] = player_times[id]

  return final




In [26]:
final_data = add_time()
#list(map(id_to_username, ids_test))
#ids_test

In [27]:
sorted = pd.Series(final_data).sort_values(ascending=False)
sorted

dopeZera          700351.00
popesquidward     553996.81
clairelynnd       427166.00
HyrulianKnight    332996.00
FFerraz1          310728.00
                    ...    
maosfx              2003.00
JoshCalamari        1907.00
kemXtrm             1780.00
Marthur              759.00
agdf_ydy             102.00
Length: 296, dtype: float64

In [28]:

def convert2(seconds):
  m, s = divmod(seconds, 60)
  h, m = divmod(m, 60)

  return str(int(h)) + ':' + str(int(m)) + ':' + str(int(s))

In [29]:
final = sorted.apply(convert2)

final.head(20)

dopeZera             194:32:31
popesquidward        153:53:16
clairelynnd          118:39:26
HyrulianKnight        92:29:56
FFerraz1              86:18:48
AntiLink              83:11:34
TrulyNaN               82:6:42
Zunar                 69:21:26
VPP                   67:49:40
SampleName_           65:35:16
Türkenheimer          64:35:22
ki9698                61:12:50
MaxSen_G               60:52:5
flanthis               54:6:34
Marco                 52:33:39
Eumeus14               47:0:25
Zeepo                 46:17:28
Severed_Speedruns     45:16:44
mrjerkyjerky          44:28:18
ADruck                40:42:29
dtype: object

In [30]:
test = json.loads(requests.get('https://www.speedrun.com/api/v1/users/1xy90mjr').text)
final['Your_Bro_Bri']


'14:56:42'

In [31]:
final['TrevPerson']

KeyError: 'TrevPerson'